# CRAWL JOB LINKS FROM CAREERVIET

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
# ====================
# 1. Cấu hình Chrome headless
# ====================
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
def get_job_links(base_url, page_start, page_end):
    job_links_list = []
    for page in range(page_start, page_end + 1):
        print(f"Current job page: {page}")
        url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR, 'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))

    return job_links_list

BASE_URL = "https://careerviet.vn/viec-lam/"

# Get job links from multiple pages (e.g., pages from 102 to 202)
job_links = get_job_links(BASE_URL, 1, 200)
# Ghi các link vào file txt
with open('Data/job_links.txt', 'w', encoding='utf-8') as f:
    for link in job_links:
        f.write(link + '\n')


Current job page: 1
Current job page: 2
Current job page: 3
Current job page: 4
Current job page: 5
Current job page: 6
Current job page: 7
Current job page: 8
Current job page: 9
Current job page: 10
Current job page: 11
Current job page: 12
Current job page: 13
Current job page: 14
Current job page: 15
Current job page: 16
Current job page: 17
Current job page: 18
Current job page: 19
Current job page: 20
Current job page: 21
Current job page: 22
Current job page: 23
Current job page: 24
Current job page: 25
Current job page: 26
Current job page: 27
Current job page: 28
Current job page: 29
Current job page: 30
Current job page: 31
Current job page: 32
Current job page: 33
Current job page: 34
Current job page: 35
Current job page: 36
Current job page: 37
Current job page: 38
Current job page: 39
Current job page: 40
Current job page: 41
Current job page: 42
Current job page: 43
Current job page: 44
Current job page: 45
Current job page: 46
Current job page: 47
Current job page: 48
C

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
# ====================
# 1. Cấu hình Chrome headless
# ====================
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
def get_job_links(base_url, page_start, page_end):
    job_links_list = []
    for page in range(page_start, page_end + 1):
        print(f"Current job page: {page}")
        url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR, 'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))

    return job_links_list

BASE_URL = "https://careerviet.vn/viec-lam/"

# Get job links from multiple pages (e.g., pages from 102 to 202)
job_links = get_job_links(BASE_URL, 201, 505)
# Ghi các link vào file txt
with open('Data/job_links.txt', 'a', encoding='utf-8') as f:
    for link in job_links:
        f.write(link + '\n')


Current job page: 201
Current job page: 202
Current job page: 203
Current job page: 204
Current job page: 205
Current job page: 206
Current job page: 207
Current job page: 208
Current job page: 209
Current job page: 210
Current job page: 211
Current job page: 212
Current job page: 213
Current job page: 214
Current job page: 215
Current job page: 216
Current job page: 217
Current job page: 218
Current job page: 219
Current job page: 220
Current job page: 221
Current job page: 222
Current job page: 223
Current job page: 224
Current job page: 225
Current job page: 226
Current job page: 227
Current job page: 228
Current job page: 229
Current job page: 230
Current job page: 231
Current job page: 232
Current job page: 233
Current job page: 234
Current job page: 235
Current job page: 236
Current job page: 237
Current job page: 238
Current job page: 239
Current job page: 240
Current job page: 241
Current job page: 242
Current job page: 243
Current job page: 244
Current job page: 245
Current jo

# PARSE JOB DETAILS

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


def fetch_job_information(job_url,headers):
    try:
        
        response = requests.get(job_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        job_name=soup.select_one("h1.title").text if soup.select_one('h1.title') else "Not specified"
        
        company_name=soup.select_one('a.employer.job-company-name').text if soup.select_one('a.employer.job-company-name') else "Not specified"
        
        location=soup.select_one('div.map p a').text if soup.select_one('div.map p a') else "Not specified"
        
        salary=soup.select_one("strong:contains('Lương') + p").text if soup.select_one("strong:contains('Lương') + p") else "Not specified"
        import re
        experience=(
            soup.select_one("strong:contains('Kinh nghiệm') + p").text.strip()
            if soup.select_one("strong:contains('Kinh nghiệm') + p")
            else "Not specified"
        )
        experience=re.sub(r'\s+', ' ', experience.strip())
        
        industry_tag = soup.select_one("strong:contains('Ngành nghề') + p")
        if industry_tag:
            industry_list = [item.get_text(strip=True) for item in industry_tag.find_all("a")]
            industry = ", ".join(industry_list) if industry_list else industry_tag.get_text(strip=True)
        else:
            industry = "Not specified"
        
        job_level=(
            soup.select_one("strong:contains('Cấp bậc') + p").text.strip()
            if soup.select_one("strong:contains('Cấp bậc') + p")
            else "Not specified"
        )
        
        employment_type=(
            soup.select_one("strong:contains('Hình thức') + p").text.strip()
            if soup.select_one("strong:contains('Hình thức') + p")
            else "Not specified"
        )
        
        welfare_elements=soup.select("ul.welfare-list li") 
        welfares=', '.join([welfare.text.strip() for welfare in welfare_elements]) if welfare_elements else "Not specified"
        
        job_description_section=soup.select("h2:contains('Mô tả Công việc') ~ *")
        job_description_list=[]
        for section in job_description_section:
            if section.name=='p' and section.text:
                job_description_list.append(section.text.strip())
            elif section.name=="ul":
                description_items = section.find_all("li")
                job_description_list.extend([item.text.strip() for item in description_items if item.text.strip()])
        job_description = '\n'.join(job_description_list) if job_description_list else "Not specified"
        
        
        job_requirements_section = soup.select("h2:contains('Yêu Cầu Công Việc') ~ *")
        job_requirements_list = []
        for section in job_requirements_section:
            if section.name == "p" and section.text.strip():
                job_requirements_list.append(section.text.strip())
            elif section.name == "ul":
                requirements_items = section.find_all("li")
                job_requirements_list.extend([item.text.strip() for item in requirements_items if item.text.strip()])
        job_requirements = '\n'.join(job_requirements_list) if job_requirements_list else "Not specified"
        
        other_info_section = soup.select_one("h3:contains('Thông tin khác') + div ul")
        
        if other_info_section:
            items = [
                re.sub(r'\s+', ' ', item.get_text(strip=True))  # Xoá khoảng trắng thừa
                for item in other_info_section.find_all("li")
            ]
            other_info = '\n'.join(items)  # Giữ xuống dòng giữa các mục
        else:
            other_info = "Not specified"
        
        # print(job_name)
        # print(company_name)
        # print(location)
        # print(salary)
        # print(experience)
        # print(industry)
        # print(job_level)
        # print(welfares)
        
        job_information = {
            "Job Name": job_name,
            "Company Name":company_name,
            "Location": location,
            "Experience": experience,
            "Job Level": job_level,
            "Industry": industry,
            "Employment Type": employment_type,
            "Welfare": welfares,
            "Job Description": job_description,
            "Job Requirements": job_requirements,
            "Other Information": other_info,
            "Salary": salary
        }
        return job_information
        
    except requests.RequestException as e:
        print(f"Error fetching data from '{job_links}': {e}")
        return None

# def crawl_job_information(job_urls):
#     job_information_list = []
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }
    
#     total_urls = len(job_urls)
#     with ThreadPoolExecutor(max_workers=30) as executor:
#         futures = [executor.submit(fetch_job_information, url, headers) for url in job_urls]
#         for i, future in enumerate(as_completed(futures), 1):
#             result = future.result()
#             if result:
#                 job_information_list.append(result)
#             # Print progress
#             print(f"Crawled {i}/{total_urls} URLs")
    
#     return pd.DataFrame(job_information_list)

def crawl_job_information(job_links):
    
    total_url=len(job_links)
    job_information_list = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    for i, job_url in enumerate(job_links, 1):
        print(f"Fetching information {i} from: {job_url}")
        result = fetch_job_information(job_url, headers)
        if result:
            job_information_list.append(result)
    return pd.DataFrame(job_information_list)

with open('Data/job_links.txt','r',encoding='utf-8') as f:
    job_links=[line.strip() for line in f]


job_links=job_links[22500:25000]
job_data=crawl_job_information(job_links)
job_data.to_csv('Data/raw_data.csv', mode='a', header=False, index=False, encoding='utf-8-sig')

Fetching information 1 from: https://careerviet.vn/vi/tim-viec-lam/development-staff-footwear-components.35C45DBC.html
Fetching information 2 from: https://careerviet.vn/vi/tim-viec-lam/chuyen-gia-an-ninh-cong-nghe.35C45D92.html
Fetching information 3 from: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-quan-he-khach-hang-ca-nhan-nam-a-bank-thang-long.35C45D61.html
Fetching information 4 from: https://careerviet.vn/vi/tim-viec-lam/truong-nhom-ban-san-pham-tin-chap-phong-ban-hang-truc-tiep-can-tho-can-tho-ta161.35C45D5B.html
Fetching information 5 from: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-pe.35C45D5D.html
Fetching information 6 from: https://careerviet.vn/vi/tim-viec-lam/quan-ly-tuan-thu.35C45D75.html
Fetching information 7 from: https://careerviet.vn/vi/tim-viec-lam/head-of-operational-excellence.35C460BE.html
Fetching information 8 from: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-tu-van-ban-hang-dong-anh-tp-ha-noi.35C45DEC.html
Fetching information 9 from: http

In [20]:
job_data

,Job Name,Company Name,Location,Experience,Job Level,Industry,Employment Type,Welfare,Job Description,Job Requirements,Other Information,Salary
0,Giám sát kinh doanh (SS) (Phú Thọ),Tổng công ty Cổ phần Bia - Rượu - Nước giải kh...,Phú Thọ,2 - 3 Năm,Trưởng nhóm / Giám sát,"Bán lẻ / Bán sỉ, Bán hàng / Kinh doanh, Tiếp t...",Nhân viên chính thức,"Laptop, Chế độ bảo hiểm, Du Lịch, Đồng phục, C...",1. Tổ chức thực thi công tác bán hàng trên thị...,Trình độ đào tạo: Tốt nghiệp Đại học.\nSử dụng...,Bằng cấp: Đại học\nĐộ tuổi: Không giới hạn tuổ...,24 Tr - 28 Tr VND
1,Chuyên Viên Kiểm Soát Xây Dựng - Alphanam Group,Alphanam Group,Hà Nội,3 - 5 Năm,Nhân viên,Xây dựng,Nhân viên chính thức,"Laptop, Chế độ bảo hiểm, Du Lịch, Chế độ thưởn...","Xây dựng kế hoạch, đề cương kiểm soát thuộc ph...","Tốt nghiệp đại học chuyên ngành xây dựng, quản...",Bằng cấp: Đại học\nĐộ tuổi: Không giới hạn tuổ...,25 Tr - 30 Tr VND
2,Cửa Hàng Trưởng Thời Trang (HCM),Công Ty Cổ Phần Đầu Tư K&G Việt Nam,Hồ Chí Minh,1 - 5 Năm,Quản lý,"Bán lẻ / Bán sỉ, Bán hàng / Kinh doanh",Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồng phục, ...","Vận hành và quản lý cửa hàng thời trang Nam, t...","- Nam/ Nữ ngoại hình khá, tuổi từ 25 – 40\n- T...",Bằng cấp: Trung cấp\nĐộ tuổi: 25 - 40\nLương: ...,12 Tr - 16 Tr VND


In [ ]:
import csv
with open('Data/raw_data.csv', 'r', encoding='utf-8-sig', newline='') as f:
    reader = csv.reader(f)
    header = next(reader, None)   # bỏ header nếu có
    row_count = sum(1 for _ in reader)

print("Số record:", row_count)


Số record: 24775
